In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Objectives
**out_data = FC_NN x in_data + Bias + noises**  
where, out_data and in_data are 512 x 1 matrix  
       FC_NN is 512 x 512 matrix         
       Bias is 512 x 1 matrix
       
Assume that we are not sure about FC_NN and Bias matrixes but we have 10,000 dataset of in_data and out_data. 

**Let auto discover these FC_NN and Bias matrixes using Adam Optimizer from TensorFlow**

In [2]:
in_data = []
out_data = []
dataset_size = 10000
FC_NN = np.random.rand(512,512)
Bias_NN = np.random.rand(512,1)-0.5
tmp = 0
for i in range(dataset_size):
    in_data.append(np.random.rand(512,1))
for i in range(dataset_size):        
    out_data.append(((np.dot(FC_NN,in_data[i])/512)- np.ones((512,1))*0.25)*100 - Bias_NN - (np.random.rand(512,1)-0.5)*0.001)
np_in_data = np.array(in_data)
np_out_data = np.array(out_data)

**Let look at np_in and np_out**

In [36]:
print(np_in_data[0,0:10])
print(np_out_data[0,0:10])

[[0.72568098]
 [0.45004314]
 [0.59351837]
 [0.76125451]
 [0.83349071]
 [0.05624794]
 [0.64722835]
 [0.19905675]
 [0.68516819]
 [0.01911071]]
[[ 0.9376107 ]
 [ 2.24300681]
 [ 0.4757808 ]
 [ 0.48933764]
 [ 0.86376941]
 [ 0.93039345]
 [-0.25180784]
 [ 0.09673591]
 [ 0.98168589]
 [ 1.17412926]]


# Setting up the model in TensorFlow

In [30]:
learning_rate = 0.001
batch_size = 1
# np.dot(Bias,np.ones((1,batch_size))).shape
mt = tf.Variable(tf.random_normal([512, 512], stddev=0.35),
                      name="weights")
Bias = tf.Variable(tf.random_normal([512, 1], stddev=0.35),
                      name="Bias")
# mt = tf.Variable(rng.randn(), name="weight")
x = tf.placeholder(tf.float32, shape=[512, batch_size])
y = tf.placeholder(tf.float32, shape=[512, batch_size])
pred = tf.subtract(((tf.matmul(mt, x)/512)- np.ones((512,batch_size))*0.25)*100,tf.matmul(Bias,np.ones((1,batch_size),dtype=np.float32)))
loss = tf.reduce_sum(tf.pow(pred-y, 2))/batch_size
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

**Save the model and weight to model_2.ckpt**

In [31]:
saver = tf.train.Saver()
display_step = 1
init_op = tf.initialize_all_variables()
with tf.Session() as sess:    
    sess.run(init_op)     
    save_path = saver.save(sess, "model_2.ckpt")
    print("initial model file created!")

initial model file created!


# Train the model
**Reload the model and train the model. The model will be saved in every 10 Epochs. In this way, we can stop training at any time and can continue from the last saved one.**

In [32]:
training_epochs = 1600
with tf.Session() as sess:    
    saver.restore(sess, "model_2.ckpt")    
    for epoch in range(training_epochs):               
        for n in range(0,np_out_data.shape[0],batch_size):
            sess.run(optimizer,feed_dict={x:np_in_data[n:n+batch_size].reshape((512,batch_size)),y:np_out_data[n:n+batch_size].reshape(512,batch_size)})

        #Display logs per epoch step
        if (epoch) % display_step == 0:
            c = 0;
            count = 0;
            for n in range(0,np_out_data.shape[0],batch_size):
                c = c + sess.run(loss,feed_dict={x:np_in_data[n:n+batch_size].reshape((512,batch_size)),y:np_out_data[n:n+batch_size].reshape(512,batch_size)})
                count = count+1
            c = c/count
            print("Epoch:", '%04d' % (epoch+1), '/%d' % (training_epochs), "cost=", "{:.9f}".format(c))
        if (epoch) % 10 == 0:
            predict = sess.run(pred,feed_dict={x:np_in_data[n:n+batch_size].reshape((512,batch_size))}).reshape(512,batch_size)
            print("abs_err: {}".format(sum(sum(abs(predict-np_out_data[n:n+batch_size].reshape(512,batch_size))))/batch_size))
            save_path = saver.save(sess, "model_2.ckpt")
            print("Model Saved!")
            
    print("Optimization Finished!")
    predict = sess.run(pred,feed_dict={x:np_in_data[0]})
    save_path = saver.save(sess, "model_2.ckpt")
    print(sum(abs(predict-np_out_data[0])))

    

INFO:tensorflow:Restoring parameters from model_2.ckpt
Epoch: 0001 /1600 cost= 104.308304871
abs_err: 178.63878095712607
Model Saved!
Epoch: 0002 /1600 cost= 21.264690510
Epoch: 0003 /1600 cost= 0.632893554
Epoch: 0004 /1600 cost= 0.251782495
Epoch: 0005 /1600 cost= 0.196057423
Epoch: 0006 /1600 cost= 0.166456029
Epoch: 0007 /1600 cost= 0.142689383
Epoch: 0008 /1600 cost= 0.125293920
Epoch: 0009 /1600 cost= 0.113547082
Epoch: 0010 /1600 cost= 0.115132071
Epoch: 0011 /1600 cost= 0.110582868
abs_err: 4.055164411738094
Model Saved!
Epoch: 0012 /1600 cost= 0.112122439
Epoch: 0013 /1600 cost= 0.122518096
Epoch: 0014 /1600 cost= 0.134032458
Epoch: 0015 /1600 cost= 0.133980675
Epoch: 0016 /1600 cost= 0.140140128
Epoch: 0017 /1600 cost= 0.140711799
Epoch: 0018 /1600 cost= 0.141401721
Epoch: 0019 /1600 cost= 0.143268176
Epoch: 0020 /1600 cost= 0.143500515
Epoch: 0021 /1600 cost= 0.141174835
abs_err: 4.402953875887256
Model Saved!
Epoch: 0022 /1600 cost= 0.141445121
Epoch: 0023 /1600 cost= 0.135

Epoch: 0204 /1600 cost= 0.142209705
Epoch: 0205 /1600 cost= 0.140013071
Epoch: 0206 /1600 cost= 0.146250791
Epoch: 0207 /1600 cost= 0.143124627
Epoch: 0208 /1600 cost= 0.144960450
Epoch: 0209 /1600 cost= 0.145033816
Epoch: 0210 /1600 cost= 0.137887537
Epoch: 0211 /1600 cost= 0.142379449
abs_err: 2.47744152980673
Model Saved!
Epoch: 0212 /1600 cost= 0.142835345
Epoch: 0213 /1600 cost= 0.142679361
Epoch: 0214 /1600 cost= 0.141481759
Epoch: 0215 /1600 cost= 0.146215274
Epoch: 0216 /1600 cost= 0.140913314
Epoch: 0217 /1600 cost= 0.140744918
Epoch: 0218 /1600 cost= 0.144809504
Epoch: 0219 /1600 cost= 0.141554943
Epoch: 0220 /1600 cost= 0.145061916
Epoch: 0221 /1600 cost= 0.139152608
abs_err: 2.2473685532882337
Model Saved!
Epoch: 0222 /1600 cost= 0.143000864
Epoch: 0223 /1600 cost= 0.140575912
Epoch: 0224 /1600 cost= 0.146219182
Epoch: 0225 /1600 cost= 0.139154145
Epoch: 0226 /1600 cost= 0.143070154
Epoch: 0227 /1600 cost= 0.142898535
Epoch: 0228 /1600 cost= 0.145432338
Epoch: 0229 /1600 co

Epoch: 0410 /1600 cost= 0.145554496
Epoch: 0411 /1600 cost= 0.140296361
abs_err: 2.1642052208422022
Model Saved!
Epoch: 0412 /1600 cost= 0.143635056
Epoch: 0413 /1600 cost= 0.145387486
Epoch: 0414 /1600 cost= 0.144172907
Epoch: 0415 /1600 cost= 0.140883686
Epoch: 0416 /1600 cost= 0.143158124
Epoch: 0417 /1600 cost= 0.140458167
Epoch: 0418 /1600 cost= 0.143959196
Epoch: 0419 /1600 cost= 0.144409051
Epoch: 0420 /1600 cost= 0.143077718
Epoch: 0421 /1600 cost= 0.140551816
abs_err: 2.1095228399766306
Model Saved!
Epoch: 0422 /1600 cost= 0.144011627
Epoch: 0423 /1600 cost= 0.142445548
Epoch: 0424 /1600 cost= 0.142329076
Epoch: 0425 /1600 cost= 0.142743641
Epoch: 0426 /1600 cost= 0.142132778
Epoch: 0427 /1600 cost= 0.141150183
Epoch: 0428 /1600 cost= 0.141512460
Epoch: 0429 /1600 cost= 0.141515422
Epoch: 0430 /1600 cost= 0.147372795
Epoch: 0431 /1600 cost= 0.140731977
abs_err: 2.107557056647278
Model Saved!
Epoch: 0432 /1600 cost= 0.144004707
Epoch: 0433 /1600 cost= 0.140749674
Epoch: 0434 /1

Epoch: 0614 /1600 cost= 0.145021943
Epoch: 0615 /1600 cost= 0.139595009
Epoch: 0616 /1600 cost= 0.146800394
Epoch: 0617 /1600 cost= 0.142710294
Epoch: 0618 /1600 cost= 0.141943460
Epoch: 0619 /1600 cost= 0.140418301
Epoch: 0620 /1600 cost= 0.144329773
Epoch: 0621 /1600 cost= 0.141307910
abs_err: 1.849604795684596
Model Saved!
Epoch: 0622 /1600 cost= 0.141845633
Epoch: 0623 /1600 cost= 0.141856125
Epoch: 0624 /1600 cost= 0.142576044
Epoch: 0625 /1600 cost= 0.141891187
Epoch: 0626 /1600 cost= 0.142439541
Epoch: 0627 /1600 cost= 0.143243097
Epoch: 0628 /1600 cost= 0.143179384
Epoch: 0629 /1600 cost= 0.143223228
Epoch: 0630 /1600 cost= 0.141332084
Epoch: 0631 /1600 cost= 0.142117505
abs_err: 2.076670507121067
Model Saved!
Epoch: 0632 /1600 cost= 0.143172953
Epoch: 0633 /1600 cost= 0.140011432
Epoch: 0634 /1600 cost= 0.140836550
Epoch: 0635 /1600 cost= 0.145320152
Epoch: 0636 /1600 cost= 0.145993779
Epoch: 0637 /1600 cost= 0.144883945
Epoch: 0638 /1600 cost= 0.141855863
Epoch: 0639 /1600 co

Epoch: 0819 /1600 cost= 0.142112274
Epoch: 0820 /1600 cost= 0.144502188
Epoch: 0821 /1600 cost= 0.144847843
abs_err: 2.0327202958384234
Model Saved!
Epoch: 0822 /1600 cost= 0.144737303
Epoch: 0823 /1600 cost= 0.146322158
Epoch: 0824 /1600 cost= 0.143839609
Epoch: 0825 /1600 cost= 0.143185080
Epoch: 0826 /1600 cost= 0.146613782
Epoch: 0827 /1600 cost= 0.142829519
Epoch: 0828 /1600 cost= 0.143342376
Epoch: 0829 /1600 cost= 0.143574022
Epoch: 0830 /1600 cost= 0.142991098
Epoch: 0831 /1600 cost= 0.144124564
abs_err: 2.0182863755158156
Model Saved!
Epoch: 0832 /1600 cost= 0.144326541
Epoch: 0833 /1600 cost= 0.141852832
Epoch: 0834 /1600 cost= 0.145121665
Epoch: 0835 /1600 cost= 0.144499891
Epoch: 0836 /1600 cost= 0.144025204
Epoch: 0837 /1600 cost= 0.142412891
Epoch: 0838 /1600 cost= 0.146195664
Epoch: 0839 /1600 cost= 0.143109669
Epoch: 0840 /1600 cost= 0.141459122
Epoch: 0841 /1600 cost= 0.145181254
abs_err: 2.1355326074581322
Model Saved!
Epoch: 0842 /1600 cost= 0.143375286
Epoch: 0843 /

Epoch: 1023 /1600 cost= 0.145804871
Epoch: 1024 /1600 cost= 0.142187737
Epoch: 1025 /1600 cost= 0.145204893
Epoch: 1026 /1600 cost= 0.142757638
Epoch: 1027 /1600 cost= 0.143339841
Epoch: 1028 /1600 cost= 0.146398869
Epoch: 1029 /1600 cost= 0.143328901
Epoch: 1030 /1600 cost= 0.145286472
Epoch: 1031 /1600 cost= 0.144606900
abs_err: 1.9968894845450251
Model Saved!
Epoch: 1032 /1600 cost= 0.143239220
Epoch: 1033 /1600 cost= 0.145023112
Epoch: 1034 /1600 cost= 0.145561775
Epoch: 1035 /1600 cost= 0.144999113
Epoch: 1036 /1600 cost= 0.143943964
Epoch: 1037 /1600 cost= 0.145317119
Epoch: 1038 /1600 cost= 0.147536209
Epoch: 1039 /1600 cost= 0.141898056
Epoch: 1040 /1600 cost= 0.146614799
Epoch: 1041 /1600 cost= 0.144619022
abs_err: 2.028318039146761
Model Saved!
Epoch: 1042 /1600 cost= 0.141338746
Epoch: 1043 /1600 cost= 0.143315985
Epoch: 1044 /1600 cost= 0.143231464
Epoch: 1045 /1600 cost= 0.140933182
Epoch: 1046 /1600 cost= 0.143585852
Epoch: 1047 /1600 cost= 0.144455369
Epoch: 1048 /1600 c

Epoch: 1228 /1600 cost= 0.146160700
Epoch: 1229 /1600 cost= 0.141776503
Epoch: 1230 /1600 cost= 0.142703460
Epoch: 1231 /1600 cost= 0.141254672
abs_err: 1.933153022271852
Model Saved!
Epoch: 1232 /1600 cost= 0.145209190
Epoch: 1233 /1600 cost= 0.143307929
Epoch: 1234 /1600 cost= 0.144813017
Epoch: 1235 /1600 cost= 0.139115717
Epoch: 1236 /1600 cost= 0.144706004
Epoch: 1237 /1600 cost= 0.145370729
Epoch: 1238 /1600 cost= 0.140929860
Epoch: 1239 /1600 cost= 0.141869488
Epoch: 1240 /1600 cost= 0.143832369
Epoch: 1241 /1600 cost= 0.141880490
abs_err: 2.0734743679801646
Model Saved!
Epoch: 1242 /1600 cost= 0.144407080
Epoch: 1243 /1600 cost= 0.145969004
Epoch: 1244 /1600 cost= 0.142400066
Epoch: 1245 /1600 cost= 0.143976603
Epoch: 1246 /1600 cost= 0.145302807
Epoch: 1247 /1600 cost= 0.141074165
Epoch: 1248 /1600 cost= 0.142534241
Epoch: 1249 /1600 cost= 0.144771413
Epoch: 1250 /1600 cost= 0.143550022
Epoch: 1251 /1600 cost= 0.141754144
abs_err: 1.975336671654184
Model Saved!
Epoch: 1252 /16

Epoch: 1432 /1600 cost= 0.145553166
Epoch: 1433 /1600 cost= 0.144705683
Epoch: 1434 /1600 cost= 0.142145599
Epoch: 1435 /1600 cost= 0.146836346
Epoch: 1436 /1600 cost= 0.144950895
Epoch: 1437 /1600 cost= 0.140485856
Epoch: 1438 /1600 cost= 0.145016429
Epoch: 1439 /1600 cost= 0.142393357
Epoch: 1440 /1600 cost= 0.142639119
Epoch: 1441 /1600 cost= 0.146931071
abs_err: 2.0478927721981712
Model Saved!
Epoch: 1442 /1600 cost= 0.144430939
Epoch: 1443 /1600 cost= 0.143509859
Epoch: 1444 /1600 cost= 0.144430661
Epoch: 1445 /1600 cost= 0.145895639
Epoch: 1446 /1600 cost= 0.143146645
Epoch: 1447 /1600 cost= 0.145392627
Epoch: 1448 /1600 cost= 0.146424309
Epoch: 1449 /1600 cost= 0.143388506
Epoch: 1450 /1600 cost= 0.142872437
Epoch: 1451 /1600 cost= 0.144249071
abs_err: 1.8086978015764215
Model Saved!
Epoch: 1452 /1600 cost= 0.145024008
Epoch: 1453 /1600 cost= 0.146905060
Epoch: 1454 /1600 cost= 0.145068500
Epoch: 1455 /1600 cost= 0.144079096
Epoch: 1456 /1600 cost= 0.144213425
Epoch: 1457 /1600 

# Check the result
**Print Predicted FC_NN matrix vs actual one**  
**Print Predicted Bias matrix vs actual one**  
**You will find that they are actually very close**  

In [33]:
with tf.Session() as sess: 
    saver.restore(sess, "model_2.ckpt")
    print(sess.run(mt)[0][0:10])
    print(FC_NN[0][0:10])
    print(sess.run(Bias)[0:10].reshape((1,10)))
    print(Bias_NN[0:10].reshape((1,10)))

INFO:tensorflow:Restoring parameters from model_2.ckpt
[0.8819622  0.5184655  0.99393314 0.47628477 0.29114076 0.5350583
 0.6360136  0.09690046 0.41262466 0.4362032 ]
[0.86698552 0.52711286 0.97601572 0.47374677 0.29985558 0.54061712
 0.63438628 0.1065537  0.41117281 0.41050468]
[[ 0.11895262 -0.1291851   0.35215402  0.14064069 -0.4164065  -0.41947207
   0.33672574  0.3827819   0.08239073 -0.43951428]]
[[ 0.11740239 -0.1293797   0.35101045  0.14199514 -0.41405645 -0.4171884
   0.33680253  0.3807979   0.0809597  -0.44137087]]
